# Roast my code: Data dumping in AiiDA

## Dumping processes

### Some examples

```shell
verdi process dump --also-raw --also-rich --include-outputs -o 914
```

#### Directories of the (almost) entire `PwBandsWorkChain`:

```shell
❯ /bin/tree -d -L 4 ./PwBandsWorkChain-914/
./PwBandsWorkChain-914/
├── 01-relax-PwRelaxWorkChain
│   ├── 01-iteration_01-PwBaseWorkChain
│   │   ├── 01-create_kpoints_from_distance (-> No outputs here)
│   │   │   ├── inputs
│   │   │   ├── node_inputs
│   │   │   └── node_outputs
│   │   └── 02-iteration_01-PwCalculation
│   │       ├── inputs
│   │       ├── node_inputs
│   │       ├── node_outputs
│   │       └── outputs
│   └── 02-iteration_02-PwBaseWorkChain
│       ├── 01-create_kpoints_from_distance
│       │   ├── inputs
│       │   ├── node_inputs
│       │   └── node_outputs
│       └── 02-iteration_01-PwCalculation
│           ├── inputs
│           ├── node_inputs
│           ├── node_outputs
│           └── outputs
├── 02-seekpath-seekpath_structure_analysis
│   ├── inputs
│   ├── node_inputs
│   │   ├── raw
│   │   └── rich
│   └── node_outputs
│       ├── explicit_kpoints
│       ├── raw
│       └── rich
├── 03-scf-PwBaseWorkChain
│   ├── 01-create_kpoints_from_distance
│   │   ├── inputs
│   │   ├── node_inputs
│   │   │   ├── raw
│   │   │   └── rich
│   │   └── node_outputs
│   │       ├── raw
│   │       └── rich
│   └── 02-iteration_01-PwCalculation
│       ├── inputs
│       ├── node_inputs
│       │   ├── pseudos
│       │   ├── raw
│       │   └── rich
│       ├── node_outputs
│       │   ├── output_band
│       │   ├── output_trajectory
│       │   ├── raw
│       │   └── rich
│       └── outputs
└── 04-bands-PwBaseWorkChain
    └── 01-iteration_01-PwCalculation
        ├── inputs
        ├── node_inputs
        │   ├── kpoints
        │   ├── pseudos
        │   ├── raw
        │   └── rich
        ├── node_outputs
        │   ├── output_band
        │   ├── output_trajectory
        │   ├── raw
        │   └── rich
        └── outputs

65 directories
```

#### The first `PwCalculation`:

```shell
❯ /bin/tree ./PwBandsWorkChain-914/01-relax-PwRelaxWorkChain/01-iteration_01-PwBaseWorkChain/
./PwBandsWorkChain-914/01-relax-PwRelaxWorkChain/01-iteration_01-PwBaseWorkChain/
├── 01-create_kpoints_from_distance
│   ├── calculation_node_attrs.yaml
│   ├── inputs
│   │   └── source_file
│   ├── node_inputs
│   │   ├── raw
│   │   │   ├── distance_attrs.yaml
│   │   │   ├── force_parity_attrs.yaml
│   │   │   └── structure_attrs.yaml
│   │   └── rich
│   │       └── Si2-594.cif
│   └── node_outputs
│       ├── raw
│       │   └── result_attrs.yaml
│       └── rich
│           └── KpointsData-921.json
└── 02-iteration_01-PwCalculation
    ├── calculation_node_attrs.yaml
    ├── inputs
    │   ├── aiida.in
    │   └── _aiidasubmit.sh
    ├── node_inputs
    │   ├── pseudos
    │   │   └── Si
    │   │       └── Si.pbesol-n-rrkjus_psl.1.0.0.UPF
    │   ├── raw
    │   │   ├── code_attrs.yaml
    │   │   ├── kpoints_attrs.yaml
    │   │   ├── parameters_attrs.yaml
    │   │   ├── pseudos
    │   │   │   └── Si_attrs.yaml
    │   │   ├── settings_attrs.yaml
    │   │   └── structure_attrs.yaml
    │   └── rich
    │       ├── KpointsData-921.json
    │       ├── qe-7.3-ioapi-pw@thor-634.yaml
    │       └── Si2-594.cif
    ├── node_outputs
    │   ├── output_band
    │   │   ├── bands.npy
    │   │   ├── kpoints.npy
    │   │   ├── occupations.npy
    │   │   └── weights.npy
    │   ├── output_trajectory
    │   │   ├── atomic_species_name.npy
    │   │   ├── cells.npy
    │   │   ├── energy_accuracy.npy
    │   │   ├── energy_ewald.npy
    │   │   ├── energy_hartree.npy
    │   │   ├── energy.npy
    │   │   ├── energy_one_electron.npy
    │   │   ├── energy_smearing.npy
    │   │   ├── energy_threshold.npy
    │   │   ├── energy_xc.npy
    │   │   ├── fermi_energy.npy
    │   │   ├── forces.npy
    │   │   ├── positions.npy
    │   │   ├── scf_accuracy.npy
    │   │   ├── scf_iterations.npy
    │   │   ├── steps.npy
    │   │   ├── stress.npy
    │   │   └── total_force.npy
    │   ├── raw
    │   │   ├── output_band_attrs.yaml
    │   │   ├── output_parameters_attrs.yaml
    │   │   ├── output_structure_attrs.yaml
    │   │   ├── output_trajectory_attrs.yaml
    │   │   └── remote_folder_attrs.yaml
    │   └── rich
    │       ├── BandsData-927.pdf
    │       ├── Si2-929.cif
    │       └── TrajectoryData-928.cif
    └── outputs
        ├── aiida.out
        ├── data-file-schema.xml
        ├── _scheduler-stderr.txt
        └── _scheduler-stdout.txt

22 directories, 55 files
```

#### `create_kpoints_from_distance`:

```shell
❯ /bin/tree ./PwBandsWorkChain-914/01-relax-PwRelaxWorkChain/01-iteration_01-PwBaseWorkChain/01-create_kpo
ints_from_distance/
./PwBandsWorkChain-914/01-relax-PwRelaxWorkChain/01-iteration_01-PwBaseWorkChain/01-create_kpoints_from_distance/
├── calculation_node_attrs.yaml
├── inputs
│   └── source_file
├── node_inputs
│   ├── raw
│   │   ├── distance_attrs.yaml
│   │   ├── force_parity_attrs.yaml
│   │   └── structure_attrs.yaml
│   └── rich
│       └── Si2-594.cif
└── node_outputs
    ├── raw
    │   └── result_attrs.yaml
    └── rich
        └── KpointsData-921.json

7 directories, 8 files
```

#### `seekpath_structure_analysis`:

```shell
❯ /bin/tree ./PwBandsWorkChain-914/02-seekpath-seekpath_structure_analysis/
./PwBandsWorkChain-914/02-seekpath-seekpath_structure_analysis/
├── calculation_node_attrs.yaml
├── inputs
│   └── source_file
├── node_inputs
│   ├── raw
│   │   ├── reference_distance_attrs.yaml
│   │   └── structure_attrs.yaml
│   └── rich
│       └── Si2-943.cif
└── node_outputs
    ├── explicit_kpoints
    │   └── kpoints.npy
    ├── raw
    │   ├── conv_structure_attrs.yaml
    │   ├── explicit_kpoints_attrs.yaml
    │   ├── parameters_attrs.yaml
    │   └── primitive_structure_attrs.yaml
    └── rich
        ├── KpointsData-947.json
        ├── Si2-948.cif
        └── Si8-949.cif

8 directories, 13 files
```

#### The actual band structure calculation:

```shell
❯ /bin/tree ./PwBandsWorkChain-914/04-bands-PwBaseWorkChain/01-iteration_01-PwCalculation/
./PwBandsWorkChain-914/04-bands-PwBaseWorkChain/01-iteration_01-PwCalculation/
├── .aiida_node_metadata.yaml
├── calculation_node_attrs.yaml
├── inputs
│   ├── .aiida
│   │   ├── calcinfo.json
│   │   └── job_tmpl.json
│   ├── aiida.in
│   └── _aiidasubmit.sh
├── node_inputs
│   ├── kpoints
│   │   └── kpoints.npy
│   ├── pseudos
│   │   └── Si
│   │       └── Si.pbesol-n-rrkjus_psl.1.0.0.UPF
│   ├── raw
│   │   ├── code_attrs.yaml
│   │   ├── kpoints_attrs.yaml
│   │   ├── parameters_attrs.yaml
│   │   ├── parent_folder_attrs.yaml
│   │   ├── pseudos
│   │   │   └── Si_attrs.yaml
│   │   ├── settings_attrs.yaml
│   │   └── structure_attrs.yaml
│   └── rich
│       ├── KpointsData-947.json
│       ├── qe-7.3-ioapi-pw@thor-634.yaml
│       └── Si2-948.cif
├── node_outputs
│   ├── output_band
│   │   ├── bands.npy
│   │   ├── kpoints.npy
│   │   ├── occupations.npy
│   │   └── weights.npy
│   ├── output_trajectory
│   │   ├── atomic_species_name.npy
│   │   ├── cells.npy
│   │   ├── positions.npy
│   │   └── steps.npy
│   ├── raw
│   │   ├── output_band_attrs.yaml
│   │   ├── output_parameters_attrs.yaml
│   │   ├── output_trajectory_attrs.yaml
│   │   └── remote_folder_attrs.yaml
│   └── rich
│       ├── BandsData-971.pdf
│       └── TrajectoryData-972.cif
└── outputs
    ├── aiida.out
    ├── data-file-schema.xml
    ├── _scheduler-stderr.txt
    └── _scheduler-stdout.txt

15 directories, 36 files
```


### Dumping of remote data


### `raw` and `rich` data dumping

- `raw` data dumping just dumps the `attributes` of every `Node` ()

## More on `raw` and `rich` data dumping


### Customization for `rich` dumping


#### Via the CLI


#### Via a config file


## Dumping collection data

- Parent folder structure: `groups`, `calculations`, `workflows`, `data`?
    - What if no `groups` present?
    - What if no organization by groups desired? -> Provide `organize_by_groups` kwarg?

- AiiDA data in hidden directories
    - This is currently the default


### Of groups


### Of the entire `storage`


## How everything plays together


## Code design: The different Dumpers

![excuse-me-sir-inheritance](/home/geiger_j/aiida_projects/verdi-profile-dump/2024-10-10-GM-dumping/figs/excuse-me-sir-inheritance.png)

![dumper-inheritance-hierarchy](/home/geiger_j/aiida_projects/verdi-profile-dump/2024-10-10-GM-dumping/figs/dumper-inheritance-hierarchy.png)
